<a href="https://colab.research.google.com/github/sanAkel/ufs_diurnal_diagnostics/blob/main/GFS/regional_diagnostics/plot_forecast_surf_fields_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install modules

In [ ]:
!pip install cartopy
!pip install cmocean
!pip install cftime

In [ ]:
!mkdir -p data_prepared

# manually upload data from RDHPCS

!ls data_prepared/

!pwd

In [ ]:
import glob as glob
import xarray as xr

import warnings
warnings.filterwarnings('ignore')

import matplotlib
#matplotlib.use('Agg')
from matplotlib import pyplot as plt
import matplotlib.ticker as mticker

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import cmocean

## User inputs

In [ ]:
exp_name = "HR5-winter"
fcst_date = '2019-12-03'# Assume 00UTC

# US North East coast
lon_s, lon_e = [-79, -74]
lat_s, lat_e = [33, 41]

cLat = -180 # central longitude (map proj)

vName = 'SST'
cMin, cMax, cMap = [-2., 32., cmocean.cm.thermal]
color_ticks = [0,4,8,12,16,20,24,28,32]
fSize=8 # text font size
DPI = 120

## Read prepared data:
- Field(s) from forecast files.
- Prepared via [this script.](https://github.com/sanAkel/ufs_diurnal_diagnostics/blob/main/GFS/regional_diagnostics/process_data.py)

In [ ]:
fName = sorted(glob.glob('/content/data_prepared/test.nc'))
print(f"Read data file:\n{fName[0]}")

In [ ]:
ds = xr.open_dataset(fName[0])
ds = ds.assign_coords({'geolon': ds['geolon'],
                       'geolat': ds['geolat']})

## Plot global map
### Note:
- By setting  `x= ''` and `y = ''` you get the right projection for tripole data.
- See [examples here for further details.](https://mom6-analysiscookbook.readthedocs.io/en/latest/notebooks/Plotting.html)

In [ ]:
subplot_kws=dict(projection=ccrs.PlateCarree(central_longitude=cLat), facecolor='grey')

plt.figure(num=1, figsize=[8,6], dpi=DPI)
im=ds[vName].plot(x='geolon', y='geolat', vmin=cMin, vmax=cMax, cmap=cMap,subplot_kws=subplot_kws, transform=ccrs.PlateCarree(),
                  add_labels=False, add_colorbar=False)

# add separate colorbar
cb = plt.colorbar(im, ticks=color_ticks, shrink=0.5, pad=0.01)
cb.ax.tick_params(labelsize=fSize)
cb.ax.set_xlabel('{}'.format(ds[vName].units), fontsize=fSize)
cb.ax.set_ylabel('{} {}'.format(vName.upper(), fcst_date))

# draw parallels/meridiens and write labels
gl = im.axes.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=2, color='gray', alpha=0.5, linestyle='--')

# adjust labels to taste
gl.top_labels = False
gl.right_labels = False
gl.ylocator = mticker.FixedLocator([-90, -60, -30, 0, 30, 60, 90])
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': fSize-2, 'color': 'black'}
gl.ylabel_style = {'size': fSize-2, 'color': 'black'}

## Same as above, but for the region of interest

In [ ]:
fig = plt.figure(num=1, figsize=[8,6], dpi=DPI)
ax = fig.add_subplot(1,1,1, projection=ccrs.PlateCarree(central_longitude=cLat))
ax.add_feature(cfeature.LAND, facecolor='grey', alpha=0.1)
ax.coastlines()
ax.set_extent([lon_s, lon_e, lat_s, lat_e], crs=ccrs.PlateCarree())

im=ds[vName].plot(x='geolon', y='geolat', vmin=5, vmax=25, cmap='Set1_r',subplot_kws=subplot_kws, transform=ccrs.PlateCarree(),
                  add_labels=False, add_colorbar=False)

# add separate colorbar
cb = plt.colorbar(im, ticks=color_ticks, shrink=0.75, pad=0.01)
cb.ax.tick_params(labelsize=fSize)
cb.ax.set_xlabel('{}'.format(ds[vName].units), fontsize=fSize)
cb.ax.set_ylabel('{} {}'.format(vName.upper(), fcst_date))

# draw parallels/meridiens and write labels
gl = im.axes.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle='--')

# adjust labels to taste
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': fSize, 'color': 'black'}
gl.ylabel_style = {'size': fSize, 'color': 'black'}